### Install & Import libraries


In [ ]:
!pip install astropy
!pip install photutils

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits

# Read fits header

In [ ]:
from astropy.io import fits
with fits.open("GROND_g_OB_ana.fits") as hdul:
    header = hdul[0].header

header

# First look

In [ ]:
fits_file = fits.open("GROND_g_OB_ana.fits")
image_data = fits_file[0].data

img = fits_file[0].data
plt.figure()
plt.imshow(img,origin='lower' ) 
plt.colorbar()
plt.show()
plt.imshow(img,origin='lower' , cmap='Greys')
plt.colorbar()
plt.show()

## Apply Log norm

In [ ]:
img = fits_file[0].data
plt.figure()
plt.imshow(img,origin='lower',norm=LogNorm() ) 
plt.colorbar()
plt.show()

## Find sources with DAO algorithm
#### https://iopscience.iop.org/article/10.1086/131977/pdf

In [ ]:
section1 = image_data[:, :]

#find mean, median, and standard deviation
from astropy.stats import sigma_clipped_stats
mean, median, std = sigma_clipped_stats(section1,sigma=8.0)
mean, median, std

In [ ]:
from photutils.detection import DAOStarFinder



# TODO!
daofind = DAOStarFinder(fwhm =  , threshold = 5.0*std)

import numpy as np
from photutils.aperture import CircularAperture



sources = daofind(section1-median) 

xpix = sources['xcentroid']
ypix = sources['ycentroid']

positions = np.transpose((xpix,ypix))
apertures = CircularAperture(positions,r=30)


plt.figure(figsize=(20,20))
plt.imshow(section1,cmap='Greys', norm=LogNorm(), origin = 'lower', interpolation='nearest') #cmap='Greys'
apertures.plot(color='blue',lw=1.5, alpha = 0.5);

In [ ]:
plt.imshow(img-median,origin='lower',norm=LogNorm())
plt.colorbar()
plt.show()

plt.imshow(img,origin='lower',norm=LogNorm() ) 
plt.colorbar()
plt.show()

## Find appropriate radial

In [ ]:
sc = sources.to_pandas()
sc['xycent'] = list(zip(sc.xcentroid, sc.ycentroid))
sc.iloc[np.where((sc.xcentroid >= 1000) & (sc.xcentroid <= 1200))]

In [ ]:
from photutils.profiles import CurveOfGrowth
cog = CurveOfGrowth(section1,sc['xycent'].values[52],radii=[i for i in range(1,70)])
cog.plot()

In [ ]:
from photutils.profiles import RadialProfile
rfp = RadialProfile(section1,sc['xycent'].values[52],radii=[i for i in range(1,70)])
rfp.plot()

### Draw aperture with selected radial

In [ ]:
# TODO!
apertures = CircularAperture(positions,r= )


plt.figure(figsize=(20,20))
plt.imshow(section1,cmap='Greys', norm=LogNorm(), origin = 'lower', interpolation='nearest') #cmap='Greys'
apertures.plot(color='blue',lw=1.5, alpha = 0.5);

### aperture_photometry

In [ ]:
# %matplotlib notebook

In [ ]:
from photutils.aperture import CircularAnnulus, CircularAperture, ApertureStats, aperture_photometry

# TODO!
annulus_aperture = CircularAnnulus(positions, r_in=, r_out= )

# plot annuluses
plt.figure(figsize=(20,20))
plt.imshow(section1, cmap='Greys', norm = LogNorm(), origin = 'lower')
apertures.plot(color='blue',lw=1.5, alpha = 0.5);
annulus_aperture.plot(color='green',lw=1.5,alpha=0.5);
plt.show()

In [ ]:
# define background
aperstats = ApertureStats(img, annulus_aperture)
bkg_mean = aperstats.mean
aperture_area = apertures.area_overlap(img)

### TODO!
total_bkg = ( )* ( ) 

# perform aperture photometry
star_data = aperture_photometry(img, apertures)




star_data['total_bkg'] = total_bkg
for col in star_data.colnames:
    star_data[col].info.format = '%.8g'
star_data.pprint()

# calibration

In [ ]:
# TODO !
zeropoint = 

extime = 


import math
magnitudes = []
for line in star_data:
    magnitudes.append(zeropoint - (2.5*math.log10(abs(line[3]-line[4])/extime)))



star_data['magnitude'] = magnitudes
star_data.pprint(max_lines = -1, max_width = -1)

In [ ]:
star_data[50:60]
